In [1]:
! pip install lightgbm

   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   --------------------- ------------------ 0.8/1.4 MB 4.2 MB/s eta 0:00:01
   ---------------------------------------- 1.4/1.4 MB 4.2 MB/s eta 0:00:00


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import lightgbm as lg

In [3]:
dataset=pd.read_csv("insurance_pre.csv")

In [4]:
dataset

,age,sex,bmi,children,smoker,charges
0,19,female,27.900,0,yes,16884.92400
1,18,male,33.770,1,no,1725.55230
2,28,male,33.000,3,no,4449.46200
3,33,male,22.705,0,no,21984.47061
4,32,male,28.880,0,no,3866.85520
...,...,...,...,...,...,...
1333,50,male,30.970,3,no,10600.54830
1334,18,female,31.920,0,no,2205.98080
1335,18,female,36.850,0,no,1629.83350
1336,21,female,25.800,0,no,2007.94500


In [5]:
dataset=pd.get_dummies(dataset, drop_first=True, dtype=int)

In [6]:
dataset

,age,bmi,children,charges,sex_male,smoker_yes
0,19,27.900,0,16884.92400,0,1
1,18,33.770,1,1725.55230,1,0
2,28,33.000,3,4449.46200,1,0
3,33,22.705,0,21984.47061,1,0
4,32,28.880,0,3866.85520,1,0
...,...,...,...,...,...,...
1333,50,30.970,3,10600.54830,1,0
1334,18,31.920,0,2205.98080,0,0
1335,18,36.850,0,1629.83350,0,0
1336,21,25.800,0,2007.94500,0,0


In [7]:
dataset.columns

Index(['age', 'bmi', 'children', 'charges', 'sex_male', 'smoker_yes'], dtype='object')

In [8]:
independent=dataset[['age', 'bmi', 'children', 'sex_male', 'smoker_yes']]

In [9]:
dependent=dataset[['charges']]

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(independent, dependent, test_size=1/3, random_state=0)

In [11]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test)

In [12]:
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMRegressor
param_grid={'boosting_type':['gbdt', 'dart', 'rf'], 'n_estimators':[10,100]}
grid=GridSearchCV(LGBMRegressor(), param_grid, refit=True, verbose=3, n_jobs=-1)
grid.fit(X_train,y_train)

Fitting 5 folds for each of 6 candidates, totalling 30 fits


C:\Users\Bagirathan\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning: 
10 fits failed out of a total of 30.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Bagirathan\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Bagirathan\anaconda3\Lib\site-packages\lightgbm\sklearn.py", line 1189, in fit
    super().fit(
  File "C:\Users\Bagirathan\anaconda3\Lib\site-packages\lightgbm\sklearn.py", line 955, in fit
    self._Booster = train(
                    ^^^^^^
  File "C:\Users\Bagirathan\anaconda3\

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000082 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 316
[LightGBM] [Info] Number of data points in the train set: 892, number of used features: 5
[LightGBM] [Info] Start training from score 13138.323530


GridSearchCV(estimator=LGBMRegressor(), n_jobs=-1,
             param_grid={'boosting_type': ['gbdt', 'dart', 'rf'],
                         'n_estimators': [10, 100]},
             verbose=3)

In [13]:
re=grid.cv_results_
grid_predictions=grid.predict(X_test)

In [14]:
from sklearn.metrics import r2_score
r_score=r2_score(y_test,grid_predictions)

In [15]:
print("The R_score value for best parameter:{}".format(grid.best_params_),r_score)

The R_score value for best parameter:{'boosting_type': 'dart', 'n_estimators': 100} 0.8707040437150442


In [16]:
table=pd.DataFrame.from_dict(re)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_boosting_type,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.233073,0.087115,0.004731,0.004254,gbdt,10,"{'boosting_type': 'gbdt', 'n_estimators': 10}",0.723918,0.690701,0.766285,0.751469,0.683186,0.723112,0.032598,3
1,0.423545,0.020123,0.006541,0.003510,gbdt,100,"{'boosting_type': 'gbdt', 'n_estimators': 100}",0.796474,0.776414,0.836671,0.821981,0.776699,0.801648,0.024191,2
2,0.067465,0.009361,0.005802,0.003633,dart,10,"{'boosting_type': 'dart', 'n_estimators': 10}",0.684963,0.649246,0.725889,0.709598,0.646785,0.683296,0.031626,4
3,0.421827,0.038679,0.003637,0.003039,dart,100,"{'boosting_type': 'dart', 'n_estimators': 100}",0.794695,0.785312,0.842867,0.833127,0.753342,0.801869,0.032675,1
4,0.009702,0.000429,0.000000,0.000000,rf,10,"{'boosting_type': 'rf', 'n_estimators': 10}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,5
5,0.010932,0.000986,0.000000,0.000000,rf,100,"{'boosting_type': 'rf', 'n_estimators': 100}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,5


In [17]:
age_input=float(input("Age:"))
bmi_input=float(input("BMI:"))
children_input=float(input("Children:"))
sex_male_input=int(input("Sex Male 0 or 1:"))
smoker_yes_input=int(input("Smoker Yes 0 or 1:"))

Age: 48
BMI: 29
Children: 3
Sex Male 0 or 1: 1
Smoker Yes 0 or 1: 0


In [18]:
Future_Prediction=grid.predict([[age_input,bmi_input,children_input,sex_male_input,smoker_yes_input]])
print("Future_Prediction={}".format(Future_Prediction))

Future_Prediction=[15766.64601657]
